In [1]:
import pandas as pd
from excel_manager import ExcelManager
from run_data_frame_calculations import RunDataFrameCalculationsForOneDataFrame
from data_frame_calculations_standard_for_gas_respiration_tests import GasComposition
from nice_functions import NiceExcelFunction


# Make a COPY before you run the script

In [2]:
path_input_excel = r"C:\Users\robbe\Downloads\Gas_production_KRA_BEP_R.Grobben_final_01_run_File.xlsx"

#make instance of excel manger
manager = ExcelManager(path_input_excel)

# load the workbook in
manager.load_workbook()

### You can see the sheet names

In [3]:
# check the sheet names
sheet_names = manager.get_sheet_names()
print(sheet_names)

['Notes', 'GT1.1', 'GT1.2', 'GT2.1', 'GT2.2', 'GT3.1', 'GT3.2', 'GT4.1', 'GT4.2', 'GT5.1', 'GT5.2', 'GT6.1', 'GT6.2', 'GT7.1', 'GT7.2', 'GT8.1', 'GT8.2']


#### input names of table if needed

In [4]:
# column_names = ['Sample ID', 'Parallel', 'Date', 'Time', 'P atm', 'P sample before gc', 'P sample after gc', 'CH4', 'CO2', 'O2', 'N2', 'Flush (1=yes; 0=no)', 'Comments', 'GC method', 'Weight', 'Date + Time', 'Day', 'Sum', 'CH4-corr', 'CO2-corr', 'O2-corr', 'N2-corr', 'Sum-corr', 'mg_bs', 'mCO2_b', 'mCH4_b', 'mO2_b', 'mN2_b', 'mCTot_b', 'mg_as', 'mCO2_a', 'mCH4_a', 'mO2_a', 'mN2_a', 'mCTot_a', 'mCTot_produced', 'O2 consumed', 'CO2 produced', 'O2 consumed_cum', 'CO2 produced_cum', 'mCTot_produced_cum', 'Cgas_DM_cum', 'PP CO2_b', 'CO2_aq [mol/m3]', 'CO2_aq [mol]', 'CO2_produced_aq', 'DIC_cum', 'Ctot_DM', 'Ratio O2/CO2']
#
basic_column_names = ['Sample ID', 'Parallel', 'Date', 'Time', 'P atm [hPa]', 'P sample before gc [hPa]', 'P sample after gc [hPa]', 'CH4 [%]', 'CO2 [%]', 'O2 [%]', 'N2 [%]', 'Flush (1=yes; 0=no)', 'Comments', 'GC method', 'Weight [g]']



##### you can check if the sheets are in the instance of the excell manager class.

In [5]:
# check if you can see the data_frame
full_sheet = manager.load_sheet_table(sheet_name="GT5.2", start_row=9)

display(full_sheet)
print(f' As you can see now the full sheet is shown')

,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Sample ID,Parallel,Date,Time,P atm,P sample before gc,P sample after gc,CH4,CO2,O2,...,CO2_a_aq,CO2_a_aq,CO2_dissolved between timesteps_aq,CO2_cum_aq,DIC_cum,Ctot_DM,Ratio O2/CO2,None,None,None
2,None,None,None,None,hPa,hPa,hPa,%,%,%,...,mol/m3,mol,mol,mol,mg C/gDW,mg C/gDW,None,None,None,None
3,GT5,2,2023-04-17 00:00:00,15:19:00,1025.4,1229.193137,None,None,None,None,...,0,0,0.000019,0.000019,0.001477,0,0,None,None,None
4,GT5,2,2023-04-19 00:00:00,15:40:00,1022.6,1103.8,1101.4,0,2.99,17.43,...,17.632435,0.001728,0.001732,0.001751,0.136762,0.239695,0.933933,None,None,None
5,GT5,2,2023-04-19 00:00:00,17:49:00,1021.4,1128.2,1125,0,0.26,20.62,...,1.57045,0.000154,-0.001574,0.000177,0.013828,None,0,None,None,None
6,GT5,2,2023-04-24 00:00:00,11:02:00,1003.2,1085.4,1082.3,0,2.79,15.69,...,16.105035,0.001578,0.001429,0.001606,0.125456,0.314259,1.005519,None,None,None
7,GT5,2,2023-04-24 00:00:00,12:15:00,1004,1104.6,1101.4,0,0.3,20.58,...,1.77605,0.000174,-0.001404,0.000202,0.015795,None,0,None,None,None
8,GT5,2,2023-04-26 00:00:00,09:27:00,1017.2,1090.1,1086,0,1.58,18.83,...,9.175923,0.000899,0.000729,0.000931,0.072712,0.3053,0.705773,None,None,None
9,GT5,2,2023-04-28 00:00:00,10:17:00,1007.1,1072.5,1069.2,0,2.29,17.33,...,12.947915,0.001269,0.000374,0.001304,0.101896,0.356933,1.262402,None,None,None


 As you can see now the full sheet is shown


### Filling th dictionary for the constants of the samples  in the instance of hte excel manager class

In [6]:
# fil in the dict with constants for the samples
sheet_names_samples = manager.get_sheet_names()
for sheet_name in sheet_names_samples:
    # loading the panda data frame for the sample
    constants_data_frame = manager.load_constants_as_data_frame(sheet_name=sheet_name, start_row=1, end_row=7, start_col=3)

    constants_data_class = manager.load_constants_as_data_class(sheet_name=sheet_name, start_row=1, end_row=7, start_col=3)

    # fill the dictionary
    manager.fill_dict_constants_data_frames(data_frame=constants_data_frame, sheets=[sheet_name])

    manager.fill_dict_constants_data_classes(data_class=constants_data_class, sheets=[sheet_name])

    print(f"loading {sheet_name} is done")

dict_constants = manager.get_dict_constants_data_frames()
dict_constants_data_classes = manager.get_dict_constants_data_classes()

loading Notes is done
loading GT1.1 is done
loading GT1.2 is done
loading GT2.1 is done
loading GT2.2 is done
loading GT3.1 is done
loading GT3.2 is done
loading GT4.1 is done
loading GT4.2 is done
loading GT5.1 is done
loading GT5.2 is done
loading GT6.1 is done
loading GT6.2 is done
loading GT7.1 is done
loading GT7.2 is done
loading GT8.1 is done
loading GT8.2 is done


# Validate Input data (TODO!)

#### running all the calculations for each data frame (sammple)

In [7]:
# Now all the data frames are read in, calculated and returned in to the excell_file
sheet_names_samples = sheet_names[1:]
for sheet_name in sheet_names_samples:
    print(f"started with sheet: {sheet_name}")
    # loading the data frame
    column_end = NiceExcelFunction.get_column_index_from_letter(column_letter="O")
    data_frame = manager.load_sheet_table_with_input_header(sheet_name=sheet_name, column_names=basic_column_names, start_row=13, end_column=column_end)

    # loading the data class with the constants for the sample
    constants_data_class = dict_constants_data_classes[sheet_name]

    # adding date and do the gas composition corrections.
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_data_frame_processor_calculations()
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_gas_composition_calculations(set_values_gas_composition_first_row=True,
                                                                                                    ch4=0, co2=0.03, o2=21.90, n2=78.07)

    # calculations for moles before and after sampling
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_mol_gases_before_and_after_sampling(Rgas=constants_data_class.Rgas,
                                                                                                           exp_temperature=constants_data_class.expTemp,
                                                                                                           volume_headspace=constants_data_class.volume_headspace)
    # correcting the mg_bs for the first measurement
    data_frame.loc[0, "mg_as"] = data_frame.loc[0, "mg_bs"]

    # run mol gas composition
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_mol_gas_composition_calculation()
    # run moles produced
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_moles_produced()
    # run cumulative calculations carbon in gas phase
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_cumulative_production_in_the_gas_phase(molar_mass_carbon=constants_data_class.molar_mass_carbon,
                                                                                                              dry_mass_sample=constants_data_class.dry_mass_sample)

    # run calculation carbon in the aqueous phase
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_carbon_in_aqueous_phase(water_volume_in_liters=constants_data_class.water_volume,
                                                                                    dry_mass_sample=constants_data_class.dry_mass_sample)

    # run results interpretations
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_results_Interpretations()

    # print to see the process
    print(f"finished calculations {sheet_name}")

    # overwrite the old data frame or fill with the new data frame which has calculations
    manager.fill_dict_panda_data_frames(data_frame=data_frame, sheets=[sheet_name])

    # print to see the process
    print(f"filled dictionary for {sheet_name}")

started with sheet: GT1.1


KeyError: 'CO2-corr [%]'

#### you can see the data frame below. Please fill the name of the tab below:

In [ ]:
sheet_tab_name = "GT1.1"


dict_data_frames = manager.get_dict_panda_data_frames()
display(dict_data_frames[sheet_tab_name])
# Set the display options
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', None)  # Automatically adjust the column width to fit the contents

# OVERWRITING the excel file. Did you made a copy before?

In [ ]:
for sheet_name in sheet_names_samples:
 # replace the table (overwrite) in the Excel file.
    data_frame = manager.get_dict_panda_data_frames()[sheet_name]
    manager.replace_table_in_specific_sheet_with_data_frame(excel_file_path=path_input_excel,
                                                            sheet_name=sheet_name,
                                                            start_row=13,
                                                            data_frame=data_frame,
                                                            header=False
                                                            )
    print(f"replaced table in Excel for {sheet_name}")